# Student activity analysis (Продолжение, 3 часть)

In [1]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from mlxtend.plotting import category_scatter, plot_decision_regions
%config IPCompleter.greedy = True

## Загрузка Датасетов

In [2]:
events_data_train = pd.read_csv('https://stepik.org/media/attachments/course/4852/event_data_train.zip')

In [3]:
submissions_data_train = pd.read_csv('https://stepik.org/media/attachments/course/4852/submissions_data_train.zip', \
                               compression ='zip')

### Данные Датасеты будут использованны для предсказания

In [4]:
events_data_test = pd.read_csv('C:\\Users\\UserOfPC\\Documents\\Основы Data Science Stepik\\Датасеты\\events_data_test.csv')

In [5]:
submission_data_test = pd.read_csv('C:\\Users\\UserOfPC\\Documents\\Основы Data Science Stepik\\Датасеты\\submission_data_test.csv')

### Предобработка и фильтрация данных (используя данные о первых двух днях активности на курсе)

In [6]:
def create_df(events_data, submissions_data, days_threshold=2):

    events_data['date'] = pd.to_datetime(events_data.timestamp, unit='s')
    submissions_data['date'] = pd.to_datetime(submissions_data.timestamp, unit='s')
    events_data['day'] = events_data.date.dt.date
    submissions_data['day'] = submissions_data.date.dt.date

    # Посчитаем число степов, которые решил каждый пользователей.
    users_events_data=events_data.pivot_table(index = 'user_id', 
                            columns='action', 
                            values='step_id', 
                            aggfunc='count',
                            fill_value=0).reset_index()

    # Посчитаем сколько данных, имеющих статус "correct" submit.
    users_scores = submissions_data.pivot_table(index = 'user_id', 
                            columns='submission_status', 
                            values='step_id', 
                            aggfunc='count',
                            fill_value=0).reset_index()

    # Таймстемп нашего порогового значения в днях:
    drop_out_threshold = days_threshold * 24 * 60 * 60

    users_data = events_data.groupby('user_id', as_index=False) \
        .agg({'timestamp': 'max'}) \
        .rename(columns={'timestamp': 'last_timestamp'})

    now = 1526772811

    users_data['is_gone_user'] = (now - users_data.last_timestamp) > drop_out_threshold

    # для того, чтобы не потерять юзеров, у которых не было ни 1 попытки в табл. users_scores
    # будем мерджить 'outer'. Т.к. по умолчанию 'inner' - только пересечения.
    users_data = users_data.merge(users_scores, on='user_id', how='outer')

    users_data = users_data.merge(users_events_data, how='outer')

    # Посчитаем число уникальных дней для каждого юзера и примерджим к Датафрейму users_data.
    users_days = events_data.groupby('user_id').day.nunique().to_frame().reset_index()

    users_data = users_data.merge(users_days, how='outer')


    return users_data

### Фильтрация данных и добавление целевой переменной passed_course 

In [7]:
def create_df_filter(users_data, filter=40):

    # Юзер успешно прошел курс, если набрал более 40 баллов (за 2 дня).
    users_data['passed_course'] = users_data.passed > filter

    # доля прошедших курс
    users_data['passed_course'].value_counts(normalize=True)

    return users_data

### Предобработка и добавление признаков

In [8]:
def preprocessing_filter(users_data, events_data, submissions_data, days_threshold=2):

    # Задача: отобрать те наблюдения, которые мы будем использовать для обучения, events data, 
    # т.е. события которые происходили с ним в течении первых 2 дней.

    user_min_time = events_data.groupby('user_id', as_index=False) \
        .agg({'timestamp': 'min'}) \
        .rename({'timestamp': 'min_timestamp'}, axis=1)

    users_data = users_data.merge(user_min_time, how='outer')

    events_data['user_time'] = events_data.user_id.map(str) + '_' + events_data.timestamp.map(str)

    # посчитаем порог
    learning_time_treshold = days_threshold * 24 * 60 * 60  

    # для каждого юзера посчитаем его 3х дневную дату (первые дни) и склеим это в строку с айди
    user_learning_time_treshold = user_min_time.user_id.map(str) + '_' + (user_min_time.min_timestamp + learning_time_treshold).map(str)

    user_min_time['user_learning_time_treshold'] = user_learning_time_treshold

    # Смерджим, outer - чтобы не потерять пропущенные значения
    events_data = events_data.merge(user_min_time[['user_id', 'user_learning_time_treshold']], how='outer') 

    # Теперь сравним строки
    # Отфильтруем данные по каждому пользователю, чтобы user_time вписывался в наши сроки
    events_data_final = events_data[events_data.user_time <= events_data.user_learning_time_treshold]

    # Сначала отфильтруем все неверные шаги всех пользователей
    submissions_data_wrong = submissions_data[(submissions_data.submission_status == 'wrong')]

    # Для каждого пользователя найдем его последний по времени шаг
    submissions_data_last = submissions_data_wrong.groupby(['user_id']) \
        .agg({'timestamp': 'max'}) \
        .rename(columns={'timestamp': 'last_timestamp'})

    # Смерджим эти две таблицы
    submissions_data_last = submissions_data_last.merge(submissions_data_wrong[['step_id', 'user_id', 'submission_status']], on='user_id', how='outer')

    # Сгруппируем по степу, посчитаем количество юзеров по каждому степу и отсортируем по убыванию.
    submissions_data_last = submissions_data_wrong.groupby(['step_id'], as_index=False) \
        .agg({'user_id': 'count'}) \
        .sort_values(by=['user_id'], ascending=False) \
        .rename(columns={'user_id': 'count_user_id'})

    # Проделаем все то же самое для ДатаСета submissions_data
    submissions_data['user_time'] = submissions_data.user_id.map(str) + '_' + submissions_data.timestamp.map(str)
    submissions_data = submissions_data.merge(user_min_time[['user_id', 'user_learning_time_treshold']], how='outer')
    submissions_data_final= submissions_data[submissions_data.user_time <= submissions_data.user_learning_time_treshold]

    return events_data_final, submissions_data_final

### Создание датасетов X и y для модели 

In [9]:
def create_train_X(events_data_final, submissions_data_final, users_data):
    
    # Возьмем число уникальных дней, которые есть в сабмитах
    X = submissions_data_final.groupby('user_id').day.nunique().to_frame().reset_index() \
        .rename(columns={'day': 'days'})

    # Посчитаем сколько степов человек попытался решить за первые дни
    steps_tried = submissions_data_final.groupby('user_id').step_id.nunique().to_frame().reset_index() \
        .rename(columns={'step_id': 'step_tried'})

    X = X.merge(steps_tried, on='user_id', how='outer')

    # Добавим по каждому пользователю кол-во правильных/неправильных сабмитов за первые 2 дня
    X = X.merge(submissions_data_final.pivot_table(index = 'user_id', 
                            columns='submission_status', 
                            values='step_id', 
                            aggfunc='count',
                            fill_value=0).reset_index())

    # Добавим еще одну переменную - доля правильных ответов
    X['correct_ratio'] = round(X.correct / (X.correct + X.wrong), 2)

    # Добавим сколько было суммарно просмотренных степов
    X = X.merge(events_data_final.pivot_table(index = 'user_id', 
                            columns='action', 
                            values='step_id', 
                            aggfunc='count',
                            fill_value=0).reset_index()[['user_id', 'viewed']], how='outer')

    # Заполним пропуски нулем
    X = X.fillna(0)

    # Добавим информацию закончил ли пользователь курс и дропнулся он или нет 
    X = X.merge(users_data[['user_id', 'passed_course', 'is_gone_user']], how='outer')

    # Отфильтруем данные для X (нам нужны кто уже либо прошел курс либо курс не прошел, но бросил его) Знак тильда - отрицание
    X = X[~((X.is_gone_user == False) & (X.passed_course ==False))]
    
    y = X.passed_course.map(int)

    X = X.drop(['passed_course', 'is_gone_user'], axis=1)

    # Сделаем в качестве индекса - user_id
    X = X.set_index(X.user_id)
    X = X.drop('user_id', axis=1)

    return X, y

Данная функция для создания итогового датасета Х для предсказания немного отличается:

In [10]:
def create_test_X(events_data_final, submissions_data_final, users_data):
    
    # Возьмем число уникальных дней, которые есть в сабмитах
    X = submissions_data_final.groupby('user_id').day.nunique().to_frame().reset_index() \
        .rename(columns={'day': 'days'})

    # Посчитаем сколько степов человек попытался решить за первые дни
    steps_tried = submissions_data_final.groupby('user_id').step_id.nunique().to_frame().reset_index() \
        .rename(columns={'step_id': 'step_tried'})

    X = X.merge(steps_tried, on='user_id', how='outer')

    # Добавим по каждому пользователю кол-во правильных/неправильных сабмитов за первые 2 дня
    X = X.merge(submissions_data_final.pivot_table(index = 'user_id', 
                            columns='submission_status', 
                            values='step_id', 
                            aggfunc='count',
                            fill_value=0).reset_index())

    # Добавим еще одну переменную - доля правильных ответов
    X['correct_ratio'] = round(X.correct / (X.correct + X.wrong), 2)

    # Добавим сколько было суммарно просмотренных степов
    X = X.merge(events_data_final.pivot_table(index = 'user_id', 
                            columns='action', 
                            values='step_id', 
                            aggfunc='count',
                            fill_value=0).reset_index()[['user_id', 'viewed']], how='outer')

    # Заполним пропуски нулем
    X = X.fillna(0)

    # Добавим информацию закончил ли пользователь курс и дропнулся он или нет 
    X = X.merge(users_data[['user_id', 'passed_course', 'is_gone_user']], how='outer')
       
    y = X.passed_course.map(int)

    X = X.drop(['passed_course', 'is_gone_user'], axis=1)

    # Сделаем в качестве индекса - user_id
    X = X.set_index(X.user_id)
    X = X.drop('user_id', axis=1)

    return X

In [11]:
def final_create_train_df(events_data, submissions_data):

    users_data_1 = create_df(events_data, submissions_data)
    users_data_2 = create_df_filter(users_data_1)
    events_data_final, submissions_data_final = preprocessing_filter(users_data_2, events_data, submissions_data)
    X, y = create_train_X(events_data_final, submissions_data_final, users_data_2)
        
    return X, y

In [12]:
X, y = final_create_train_df(events_data_train, submissions_data_train)

In [13]:
def final_create_test_df(events_data, submissions_data):

    users_data_1 = create_df(events_data, submissions_data)
    users_data_2 = create_df_filter(users_data_1)
    events_data_final, submissions_data_final = preprocessing_filter(users_data_2, events_data, submissions_data)
    X = create_test_X(events_data_final, submissions_data_final, users_data_2)
        
    return X

In [14]:
X_test_final = final_create_test_df(events_data_test, submission_data_test)

In [15]:
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.3, random_state=42)

### Обучение модели RandomForestClassifier

In [46]:
clf_rf = RandomForestClassifier(class_weight='balanced', random_state=42)

In [47]:
parametrs = {'n_estimators': [7, 40, 100], 
             'max_depth': [2, 5, 7, 10],
             'min_samples_leaf' : [10],
             'min_samples_split': [10]}

In [48]:
grid_search_cv_clf_rf = GridSearchCV(clf_rf, parametrs, cv=5)

In [49]:
grid_search_cv_clf_rf.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=42),
             param_grid={'max_depth': [2, 5, 7, 10], 'min_samples_leaf': [10],
                         'min_samples_split': [10],
                         'n_estimators': [7, 40, 100]})

In [50]:
grid_search_cv_clf_rf.best_params_

{'max_depth': 10,
 'min_samples_leaf': 10,
 'min_samples_split': 10,
 'n_estimators': 40}

In [51]:
best_clf_rf = grid_search_cv_clf_rf.best_estimator_

In [52]:
y_pred = best_clf_rf.predict(X_test)
print('rf')
print(f'Accuracy = {round(accuracy_score(y_test, y_pred), 2)}')
print(f'Precision = {round(precision_score(y_test, y_pred), 2)}')
print(f'Recall = {round(recall_score(y_test, y_pred), 2)}')
print(f'F1-score = {round(f1_score(y_test, y_pred), 2)}')

rf
Accuracy = 0.79
Precision = 0.53
Recall = 0.79
F1-score = 0.63


In [53]:
y_pred = best_clf_rf.predict(X_test)

In [54]:
y_predicted_prob = best_clf_rf.predict_proba(X_test)
roc_score = roc_auc_score(y_test, y_predicted_prob[:, 1])
print(f'roc_score = {round(roc_auc_score(y_test, y_predicted_prob[:, 1]), 2)}')

roc_score = 0.87


### Итоги и финальное предсказание сможет ли пользователь успешно закончить онлайн курс Анализ данных в R.

Видим, что теперь, после финальной предобработки данных, отбора первых двух дней активности по каждому пользователю и фильтрации по баллам (выбранный порог для меток классов - 40 баллов), новая модель RandomForestClassifier с подобранными парметрами выдает более качественные результирующие метрики. Данная модель нас устраивает и может быть проведено финальное предсказание на данных events_data_test и  submission_data_test.

In [55]:
ypred_prob_final = best_clf_rf.predict_proba(X_test_final)

Получим финальную таблицу с предсказаниями вероятностей покинет ли студент курс или закончит его.

In [56]:
result = X_test_final.reset_index()
result = pd.DataFrame(result.iloc[:, 0])
result['is_gone'] = ypred_prob_final[:, 1]
result

,user_id,is_gone
0,12,0.490399
1,13,0.999528
2,21,0.991330
3,35,0.661330
4,45,0.563037
...,...,...
6179,26785,0.238893
6180,26791,0.109798
6181,26795,0.109798
6182,26796,0.288485


Сохраним результаты в файл

In [31]:
# result.to_csv('result_of_contest.csv', index=False)